# **Prompt Templates**

Now let's check the formal definition of prompt templete:

**A LangChain prompt template is a class containing elements you typically need for a Large Language Model (LLM) prompt. At a minimum, these are:.**



- `A natural language string that will serve as the prompt:` This can be a simple text string, or, for prompts consisting of dynamic content, an f-string or docstring containing placeholders that represent variables.
- `Formatting instructions (optional)`, that specify how dynamic content should appear in the prompt, i.e., whether it should be italicized, capitalized, etc.
- `Input parameters (optional)` that you pass into the prompt class to provide instructions or context for generating prompts. These parameters influence the content, structure, or formatting of the prompt. But oftentimes they’re variables for the placeholders in the string, whose values resolve to produce the final string that goes into the LLM through an API call as the prompt.

> ### Let's also take a look on what LangChain say about prompt templates:

> Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.


LangChain provides several types of prompt templates to handle different scenarios and input formats. Here are their names with one-line explanations:

### 1. PromptTemplate
A general-purpose template for creating customizable prompts with dynamic input variables.
### 2. FewShotPromptTemplate
Used for "few-shot learning" by including examples in the prompt to guide the language model.
### 3. ChatPromptTemplate
Designed for constructing structured conversational prompts with multi-turn interactions (e.g., system, user, and AI messages).
### 4. OutputParser
Not strictly a template but used to parse and format model outputs into structured formats, often combined with prompt templates..

----

### **String PromptTemplates**
These prompt templates are used to format a single string, and generally are used for simpler inputs. For example, a common way to construct and use a PromptTemplate is as follows:

In [1]:
from langchain_core.prompts import PromptTemplate

# In between the paranthesis we will give our prompt in string
prompt_template = PromptTemplate.from_template("Please Write an essay about {topic}.")

## After initializing out prompt_template we have to invoke the prompt_template

prompt_template.invoke({'topic': 'school'})

StringPromptValue(text='Please Write an essay about school.')

In [2]:
template = "Translate this to {language}: {text}"

prompt = PromptTemplate(input_variables=["language", "text"], template=template)

formatted_prompt = prompt.format(language="Spanish", text="Hello!")
print(formatted_prompt)

Translate this to Spanish: Hello!


### **ChatPromptTemplates**
> These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves. For example, a common way to construct and use a ChatPromptTemplate is as follows:

- The ChatPromptTemplate class focuses on the conversation flow between a user and an AI system, and provides instructions or requests for roles like user, system, assistant, and others (the exact roles you can use will depend on your LLM model).

- Such roles give deeper context to the LLM and elicit better responses that help the model grasp the situation more holistically. System messages in particular provide implicit instructions or set the scene, informing the LLM of expected behavior

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate(
    [
    ('system', 'You are a helpful assistant for Q/A tasks'),
    ("user", '{user_input}'),
    ]
)

prompt_template

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant for Q/A tasks'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Tell the coordinates of country pakistan'), additional_kwargs={})])

In [4]:
prompt_template.invoke({'user_input': 'Tell me the capital of Pakistan'})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant for Q/A tasks', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell the coordinates of country pakistan', additional_kwargs={}, response_metadata={})])

In [5]:
chat_template = ChatPromptTemplate([
    ("system", "You are a knowledgeable AI assistant. You are called {name}."),
    ("user", "Hi, what's the weather like today?"),
    ("ai", "It's sunny and warm outside."),
    ("user", "{user_input}"),
])

In [6]:
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a knowledgeable AI assistant. You are called {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Hi, what's the weather like today?"), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="It's sunny and warm outside."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])

In [7]:
messages = chat_template.format_messages(name="Joker", user_input="Can you tell me a joke?")


In [9]:
messages

[SystemMessage(content='You are a knowledgeable AI assistant. You are called Joker.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi, what's the weather like today?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's sunny and warm outside.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you tell me a joke?', additional_kwargs={}, response_metadata={})]

## Let's take a role like this:

- `System` for a system chat message setting the stage (e.g., “ You are a knowledgeable historian ”).
- `User`, which contains the user’s specific historical question.
- `AI`, which contains the LLM’s preliminary response or follow-up question.

Once the template object is instantiated, you can use it to generate chat prompts by replacing the placeholders with actual content.

This prompt sets the context, provides the user's question, and typically leaves the AI response blank for the LLM to generate.

### **FewShotPromptTemplate**

> We'll learn how to create a simple prompt template that provides the model with example inputs and outputs when generating. Providing the LLM with a few such examples is called few-shotting, and is a simple yet powerful way to guide generation and in some cases drastically improve model performance.
>
>A few-shot prompt template can be constructed from either a set of examples, or from an Example Selector class responsible for choosing a subset of examples from the defined set.

`FewShotPromptTemplate` is Often a more useful form of prompting than sending a simple string with a request or question is to include several examples of outputs you want.

In [10]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [11]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template="Question: {question}/{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)